## Proceso 1: Extracción de canciones y cantantes

In [1]:
import numpy as np
from extraccion import webscraping
import sys
sys.path.append(sys.path[0] + '/..')
from logica import cancion_logica, sitio_logica

import time
from cajaBlanca import auditoria

EXTRACCION_CANCION = 'Extracción Canción Cantante Sitio'
tiempo_inicial = time.time()

# obtener contenido de canciones más escuchadas de la página top 100 del mes de enero portaldemusica.es
URL = 'https://www.elportaldemusica.es/lists/top-100-canciones/2022/1'
contenedor = 'w1'
divPrincipal = 'item'
elementosHtml = np.array([
   ['p', 'single-list-entry-rank-position'], 
   ['div', 'name'],
   ['div', 'related'] 
])
elementosExtraidos = webscraping.obtenerElementos(URL, contenedor, divPrincipal, elementosHtml)

# registra canciones
lista_canciones = cancion_logica.insertarCancionCantante(elementosExtraidos)

# registra sitio web
sitio_logica.insertarSitio(URL, lista_canciones)

# registra pista del proceso de extracción
auditoria.registrarPistaProceso([EXTRACCION_CANCION, tiempo_inicial, [auditoria.ESTADO_COMPLETADO, 'sitio:www.elportaldemusica.es', 'top:100', '2022-01']])

## Proceso 2: Extracción de Letras de canciones

In [2]:
'''
Obtiene letra de canciones
'''
from logica import cancion_logica, letras_logica

import time
from cajaBlanca import auditoria

PROCESO_PRINCIPAL = 'Extracción Letras General'
tiempo_inicial = time.time()

# variables utilizados como contadores de exito y fracaso
exito = 0
fracaso = 0

# obtener canciones pendientes si letras registradas
canciones_pendientes = cancion_logica.obtenerCancionesPorEstado(cancion_logica.ESTADO_SIN_LETRA)

for cancion_pendiente in canciones_pendientes:
    # obtiene el nombre de la canción y del cantante de la base de datos que no tiene letras
    nombre_cancion = cancion_pendiente['cancion']
    nombre_cantante = cancion_pendiente['cantante']
   
    # obtiene letra de canción desde varios conetnidos web
    tiempo_inicial_lyrics = time.time()
    letra_cancion = letras_logica.obtenerLetraConLyrics(nombre_cancion, nombre_cantante)
    if letra_cancion == '':
        auditoria.registrarPistaProceso(['Extracción Letra Lyrics', tiempo_inicial_lyrics, [auditoria.RESULTADO_FRACASO]])
        tiempo_inicial_letras4u = time.time()
        letra_cancion = letras_logica.obtenerLetraConLetras4u(nombre_cancion, nombre_cantante)
        if letra_cancion == '':
            auditoria.registrarPistaProceso(['Extracción Letra Letras4u', tiempo_inicial_letras4u, [auditoria.RESULTADO_FRACASO]])
            tiempo_inicial_yaletras = time.time()
            letra_cancion = letras_logica.obtenerLetraConYaLetras(nombre_cancion, nombre_cantante)
            if letra_cancion == '':
                auditoria.registrarPistaProceso(['Extracción Letra Yaletras', tiempo_inicial_yaletras, [auditoria.RESULTADO_FRACASO]])
                tiempo_inicial_cancioneros = time.time()
                letra_cancion = letras_logica.obtenerLetraConCancioneros(nombre_cancion, nombre_cantante)
                if letra_cancion == '':
                    auditoria.registrarPistaProceso(['Extracción Letra Cancioneros', tiempo_inicial_cancioneros, [auditoria.RESULTADO_FRACASO]])
                    tiempo_inicial_google = time.time()
                    letra_cancion = letras_logica.obtenerLetraConGoogle(nombre_cancion, nombre_cantante)
                    if letra_cancion == '':
                        auditoria.registrarPistaProceso(['Extracción Letra Google', tiempo_inicial_google, [auditoria.RESULTADO_FRACASO]])
                    else:
                        auditoria.registrarPistaProceso(['Extracción Letra Google', tiempo_inicial_google, [auditoria.RESULTADO_EXITO]])
                else:
                    auditoria.registrarPistaProceso(['Extracción Letra Cancioneros', tiempo_inicial_cancioneros, [auditoria.RESULTADO_EXITO]])
            else:
                auditoria.registrarPistaProceso(['Extracción Letra Yaletras', tiempo_inicial_yaletras, [auditoria.RESULTADO_EXITO]])
        else:
            auditoria.registrarPistaProceso(['Extracción Letra Letras4u', tiempo_inicial_letras4u, [auditoria.RESULTADO_EXITO]])

    else:
        auditoria.registrarPistaProceso(['Extracción Letra Lyrics', tiempo_inicial_lyrics, [auditoria.RESULTADO_EXITO]])

      
    # actualiza la letra en la colección de la canción
    if letra_cancion == '':
        fracaso = fracaso + 1
        cancion_logica.actualizarCancionNoEncontrado(cancion_pendiente)
    else:
        exito = exito + 1
        cancion_logica.actualizarCancionPendiente(cancion_pendiente, letra_cancion)

# registra pista del proceso de extracción
total = exito + fracaso
auditoria.registrarPistaProceso([PROCESO_PRINCIPAL, tiempo_inicial, [auditoria.ESTADO_COMPLETADO, 'total:' + str(total), 'exito:' + str(exito), 'fracaso:' + str(fracaso)]])

print('fin de la extracción de letras')

fin de la extracción de letras


## Proceso 3: Preprocesamiento de las letras extraidas con webscrapping

In [3]:
'''
Preprocesamiento de letras de canciones
'''
from logica import cancion_logica
from preprocesamiento import tokenizacion, depuracion
import time
from cajaBlanca import auditoria

PROCESO_PRINCIPAL = 'Preprocesamiento General'

tiempo_inicial = time.time()

# obtiene canciones con letras para su preprocesamiento de texto
canciones = cancion_logica.obtenerCancionesPorEstado(cancion_logica.ESTADO_CON_LETRA)
total = 0
for cancion in canciones:
   total = total + 1
   letra = cancion['letra']
   letra_tokenizado = tokenizacion.tokenizarConExpresionesRegulares(letra)
   letra_depurada = depuracion.eliminarPalabrasVacias(letra_tokenizado)
   cancion_logica.actualizarCancionPreprocesado(cancion, letra_depurada)
               
auditoria.registrarPistaProceso([PROCESO_PRINCIPAL, tiempo_inicial, [auditoria.ESTADO_COMPLETADO, 'total letras procesadas: ' + str(total)]])
  